In [1]:
!pip install scikit-learn

You should consider upgrading via the '/Users/cristovalneosasono/DataScience_Foundations/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer

In [3]:
# Pandas display configuration
pd.set_option("display.max_rows", None)  
pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 2)

In [4]:
nov22_companies_ds = pd.read_csv("November2022_Unicorn_Companies.csv")
mar22_companies_ds = pd.read_csv("March2022_Unicorn_Companies.csv")

In [5]:
# Inspecting dataset 1 with df.head()
nov22_companies_ds.head()

,Unnamed: 0,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors
0,0,ByteDance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,1,SpaceX,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,2,SHEIN,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,3,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,4,Canva,$40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [6]:
# Inspecting dataset 2 with df.head()
mar22_companies_ds.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


In [7]:
# Inspecting dataset 1 with df.info()
nov22_companies_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1201 entries, 0 to 1200
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1201 non-null   int64 
 1   Company           1201 non-null   object
 2   Valuation ($B)    1201 non-null   object
 3   Date Joined       1201 non-null   object
 4   Country           1201 non-null   object
 5   City              1183 non-null   object
 6   Industry          1201 non-null   object
 7   Select Investors  1200 non-null   object
dtypes: int64(1), object(7)
memory usage: 75.2+ KB


In [8]:
# Inspecting dataset 1 with df.info()
nov22_companies_ds.describe()

,Unnamed: 0
count,1201.00
mean,600.00
std,346.84
min,0.00
25%,300.00
50%,600.00
75%,900.00
max,1200.00


In [9]:
# Merging [nov22_companies_ds] and [mar22_companies_ds] datasets together based on the "Company" column

merged_df = pd.merge(nov22_companies_ds, mar22_companies_ds, on="Company", how="outer")
merged_df.head(5)

,Unnamed: 0,Company,Valuation ($B),Date Joined_x,Country,City_x,Industry_x,Select Investors_x,Valuation,Date Joined_y,Industry_y,City_y,Country/Region,Continent,Year Founded,Funding,Select Investors_y
0,0.0,ByteDance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,SpaceX,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",100.0,2012-12-01,Other,Hawthorne,United States,North America,2002.0,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,2.0,SHEIN,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China...",100.0,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008.0,$2B,"Tiger Global Management, Sequoia Capital China..."
3,3.0,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",95.0,2014-01-23,FinTech,San Francisco,United States,North America,2010.0,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,4.0,Canva,$40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",40.0,2018-01-08,Internet software & services,Surry Hills,Australia,Oceania,2012.0,$572M,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [10]:
# Cleaning up the data 

# Dropping unecessary columns
merged_df.drop(columns="Unnamed: 0", inplace=True)

# Merging City
merged_df['City'] = merged_df['City_x'].combine_first(merged_df['City_y'])
merged_df.drop(columns=['City_x', 'City_y'], inplace=True)

# Merging Valutation_($B)
merged_df['Valuation($B)'] = merged_df["Valuation ($B)"].combine_first(merged_df['Valuation'])
merged_df.drop(columns=["Valuation ($B)", 'Valuation'], inplace=True)

# Merging DateJoined
merged_df['DateJoined'] = merged_df["Date Joined_x"].combine_first(merged_df["Date Joined_y"])
merged_df.drop(columns=["Date Joined_x", "Date Joined_y"], inplace=True)

# Merging Industry
merged_df['Industry'] = merged_df['Industry_x'].combine_first(merged_df['Industry_y'])
merged_df.drop(columns=['Industry_x', 'Industry_y'], inplace=True)

# Merging Country/Region
merged_df["Region"] = merged_df['Country'].combine_first(merged_df['Country/Region'])
merged_df.drop(columns=['Country', 'Country/Region'], inplace=True)

# Merging SelectInvestors
merged_df['SelectInvestors'] = merged_df["Select Investors_x"].combine_first(merged_df["Select Investors_y"])
merged_df.drop(columns=["Select Investors_x", "Select Investors_y"], inplace=True)

# Renaming for neater display and better clarification
merged_df.rename(columns={'Region': 'Country'}, inplace=True)
merged_df.rename(columns={"Year Founded": 'YearFounded'}, inplace=True)

# Changing the datatype of YearFounded from float to an int
merged_df['YearFounded'] = pd.to_numeric(merged_df['YearFounded'], errors='coerce').astype('Int64')

In [11]:
merged_df.head(5)

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors
0,ByteDance,NaN,<NA>,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,North America,2002,$7B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,Asia,2008,$2B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,North America,2010,$2B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG"
4,Canva,Oceania,2012,$572M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [12]:
merged_df.tail()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors
1236,iFood,South America,2011,$592M,Osasco,1.0,2018-11-13,"Supply chain, logistics, & delivery",Brazil,"Movile, Just Eat, Naspers"
1237,Newlink Group,Asia,2016,$655M,Beijing,1.0,2020-07-10,E-commerce & direct-to-consumer,China,"JOY Capital, NIO Capital, Blueflame Capital"
1238,Orbbec Technology,Asia,2013,$200M,Shenzhen,1.0,2018-05-21,Hardware,China,"R-Z Capital, Green Pine Capital Partners, SAIF..."
1239,SoundHound,North America,2005,$215M,Santa Clara,1.0,2018-05-03,Artificial intelligence,United States,"Tencent Holdings, Walden Venture Capital, Glob..."
1240,SoundHound,North America,2005,$215M,Santa Clara,1.0,2018-05-03,Other,United States,Tencent Holdings


In [13]:
# merged_df.rename(columns={"Date Joined_x": 'Date_Joined', 
#                           "City_x":'City', 
#                           "Industry_x":'Industry', 
#                           "Select Investors_x":'Select_Investors',
#                           "Valuation ($B)": 'Valuation_($B)',
#                           "Country":'Country/Region', 
#                           "Year Founded":'Year_Founded'},
#                            inplace=True)

In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 1240
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1241 non-null   object
 1   Continent        1077 non-null   object
 2   YearFounded      1077 non-null   Int64 
 3   Funding          1077 non-null   object
 4   City             1223 non-null   object
 5   Valuation($B)    1241 non-null   object
 6   DateJoined       1241 non-null   object
 7   Industry         1241 non-null   object
 8   Country          1241 non-null   object
 9   SelectInvestors  1241 non-null   object
dtypes: Int64(1), object(9)
memory usage: 107.9+ KB


In [15]:
# Check duplicates again
has_duplicates = merged_df['Company'].duplicated().any()

if has_duplicates:
    print("The 'Company' column has duplicate values.")
else:
    print("The 'Company' column does not have duplicate values.")

The 'Company' column has duplicate values.


In [16]:
# Find and display the rows with duplicate values in the "Company" column
duplicate_rows = merged_df[merged_df['Company'].duplicated(keep=False)]
duplicate_rows

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors
42,Bolt,Europe,2013,$1B,San Francisco,$11,10/8/2021,Fintech,United States,"Activant Capital, Tribe Capital, General Atlantic"
43,Bolt,Europe,2013,$1B,Tallinn,$8.4,5/29/2018,Auto & transportation,Estonia,"Didi Chuxing, Diamler, TMT Investments"
210,Branch,North America,2014,$667M,Redwood City,$4,9/10/2018,Mobile & telecommunications,United States,"New Enterprise Associates, Pear, Cowboy Ventures"
211,Branch,North America,2014,$667M,Columbus,$1.05,3/1/2022,Fintech,United States,"American Family Ventures, Greycroft, SignalFire"
529,ZocDoc,North America,2007,$374M,New York,$1.8,8/20/2015,Health,United States,"Founders Fund, Khosla Ventures, Goldman Sachs"
530,ZocDoc,North America,2007,$374M,New York,$1.8,8/20/2015,Health,United States,"Founders Fund, Khosla Ventures, Goldman Sachs"
664,Fabric,North America,2015,$294M,Bellevue,$1.5,2/24/2022,E-commerce & direct-to-consumer,United States,"Redpoint Ventures, Norwest Venture Partners, S..."
665,Fabric,North America,2015,$294M,New York,$1,10/26/2021,"Supply chain, logistics, & delivery",United States,"Innovation Endeavors, Aleph, Temasek"
784,BrewDog,Europe,2007,$233M,Aberdeen,$1.24,4/10/2017,Consumer & retail,United Kingdom,"TSG Consumer Partners, Crowdcube"
785,BrewDog,Europe,2007,$233M,Aberdeen,$1.24,4/10/2017,Consumer & retail,United Kingdom,"TSG Consumer Partners, Crowdcube"


In [17]:
# Removing duplicate values by keeping the first instance of the duplicate
merged_df = merged_df.drop_duplicates(subset='Company', keep='first')

In [18]:
# Check duplicates again
has_duplicates = merged_df['Company'].duplicated().any()

if has_duplicates:
    print("The 'Company' column has duplicate values.")
else:
    print("The 'Company' column does not have duplicate values.")

The 'Company' column does not have duplicate values.


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235 entries, 0 to 1239
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1235 non-null   object
 1   Continent        1071 non-null   object
 2   YearFounded      1071 non-null   Int64 
 3   Funding          1071 non-null   object
 4   City             1217 non-null   object
 5   Valuation($B)    1235 non-null   object
 6   DateJoined       1235 non-null   object
 7   Industry         1235 non-null   object
 8   Country          1235 non-null   object
 9   SelectInvestors  1235 non-null   object
dtypes: Int64(1), object(9)
memory usage: 107.3+ KB


In [20]:
merged_df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239],
           dtype='int64', length=1235)

In [21]:
# Listing column_names to check missing values
column_names = merged_df.columns.tolist()
print(column_names)

['Company', 'Continent', 'YearFounded', 'Funding', 'City', 'Valuation($B)', 'DateJoined', 'Industry', 'Country', 'SelectInvestors']


In [22]:
# Displaying missing values
Total_values = 1235
print(f"Total values: {Total_values}")
for column in column_names:
    missing_count = merged_df[column].isna().sum()
    print(f"Missing values in '{column}' column: {missing_count} | Percentage of missing values: {(missing_count/Total_values) * 100:0.2f} %")

Total values: 1235
Missing values in 'Company' column: 0 | Percentage of missing values: 0.00 %
Missing values in 'Continent' column: 164 | Percentage of missing values: 13.28 %
Missing values in 'YearFounded' column: 164 | Percentage of missing values: 13.28 %
Missing values in 'Funding' column: 164 | Percentage of missing values: 13.28 %
Missing values in 'City' column: 18 | Percentage of missing values: 1.46 %
Missing values in 'Valuation($B)' column: 0 | Percentage of missing values: 0.00 %
Missing values in 'DateJoined' column: 0 | Percentage of missing values: 0.00 %
Missing values in 'Industry' column: 0 | Percentage of missing values: 0.00 %
Missing values in 'Country' column: 0 | Percentage of missing values: 0.00 %
Missing values in 'SelectInvestors' column: 0 | Percentage of missing values: 0.00 %


In [23]:
# Checking NaN values in DataFrame
is_nan = pd.isna(merged_df['Continent'])
nan_valuation_df = merged_df[is_nan]
nan_valuation_df

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors
0,ByteDance,NaN,<NA>,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S..."
21,Yuanqi Senlin,NaN,<NA>,NaN,Beijing,$15,3/1/2020,Consumer & retail,China,"Sequoia Capital China, Longfor Capitalm, Gaoro..."
32,Bitmain Technologies,NaN,<NA>,NaN,Beijing,$12,7/6/2018,Hardware,China,"Coatue Management, Sequoia Capital China, IDG ..."
45,Weilong,NaN,<NA>,NaN,Luohe,$10.88,5/8/2021,Consumer & retail,China,"Tencent Holdings, Hillhouse Capital Management..."
55,KuCoin,NaN,<NA>,NaN,Victoria,$10,5/10/2022,Fintech,Seychelles,"IDG Capital, Matrix Partners China, Jump Crypto"
93,We Doctor,NaN,<NA>,NaN,Hangzhou,$7,9/22/2015,Health,China,"Tencent, Morningside Group"
122,RELEX,NaN,<NA>,NaN,Helsinki,$5.7,2/17/2022,"Supply chain, logistics, & delivery",Finland,"Blackstone, Technology Crossover Ventures, Sum..."
123,The Boring Company,NaN,<NA>,NaN,Pflugerville,$5.7,4/20/2022,Other,United States,"VY Capital, 8VC, Craft Ventures"
151,VerSe Innovation(Dailyhunt),NaN,<NA>,NaN,Bengaluru,$5,12/22/2020,Internet software & services,India,"Falcon Edge Capital, Omidyar Network, Sequoia ..."
161,SonarSource,NaN,<NA>,NaN,Geneva,$4.7,4/26/2022,Internet software & services,Switzerland,"Advent International,General Catalyst,Insight ..."


In [24]:
# Pivoted the df to display the valuation of each company categorized by Industry
# pivoted_df = merged_df.pivot(index='Company', columns='Industry', values='Valuation($B)')
# pivoted_df

# I feel this is a bit useless to my current use, so I disabled it for now

In [25]:
# Melted the df to display the valuation of each company
filtered_df = merged_df[['Company', 'Valuation($B)']]
melted_df = filtered_df.melt(id_vars='Company', var_name='Attribute', value_name='Value')
melted_df

,Company,Attribute,Value
0,ByteDance,Valuation($B),$140
1,SpaceX,Valuation($B),$127
2,SHEIN,Valuation($B),$100
3,Stripe,Valuation($B),$95
4,Canva,Valuation($B),$40
5,Checkout.com,Valuation($B),$40
6,Instacart,Valuation($B),$39
7,Databricks,Valuation($B),$38
8,Revolut,Valuation($B),$33
9,Epic Games,Valuation($B),$31.5


In [26]:
# Clean "DateJoined" and create "YearJoined" column
merged_df['YearJoined'] = merged_df['DateJoined'].str.extract(r'(\d{4})').astype(int)
merged_df.head()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined
0,ByteDance,NaN,<NA>,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S...",2017
1,SpaceX,North America,2002,$7B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012
2,SHEIN,Asia,2008,$2B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China...",2018
3,Stripe,North America,2010,$2B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG",2014
4,Canva,Oceania,2012,$572M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat...",2018


In [27]:
# Defining "YearsToUnicorn" column by applying a custom function to calculate 
def calculate_years_to_unicorn(row):
    return row['YearJoined'] - row['YearFounded']

# Apply the custom function to the DataFrame using df.apply()
merged_df['YearsToUnicorn'] = merged_df.apply(calculate_years_to_unicorn, axis=1)
merged_df.head()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn
0,ByteDance,NaN,<NA>,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S...",2017,<NA>
1,SpaceX,North America,2002,$7B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012,10
2,SHEIN,Asia,2008,$2B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China...",2018,10
3,Stripe,North America,2010,$2B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG",2014,4
4,Canva,Oceania,2012,$572M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat...",2018,6


In [28]:
# Boolean mask for rows where the "Country" column is equal to "Indonesia"
indonesia_mask = merged_df['Country'] == 'Indonesia'

indonesia_df = merged_df[indonesia_mask]

indonesia_df.head(10)

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn
15,J&T Express,Asia,2015,$5B,Jakarta,$20,4/7/2021,"Supply chain, logistics, & delivery",Indonesia,"Hillhouse Capital Management, Boyu Capital, Se...",2021,6
286,Traveloka,Asia,2012,$1B,Jakarta,$3,1/1/2017,Travel,Indonesia,"Global Founders Capital, East Ventures, Expedi...",2017,5
498,Akulaku,Asia,2014,$438M,Jakarta,$2,2/15/2022,E-commerce & direct-to-consumer,Indonesia,"DCM Ventures, IDG Capital, Siam Commercial Bank",2022,8
853,DANA,NaN,<NA>,NaN,Jakarta,$1.13,8/10/2022,Fintech,Indonesia,"Ant Group, Lazada, Sinar Mas Indonesia",2022,<NA>
1090,Xendit,Asia,2014,$215M,Jakarta,$1,9/14/2021,Fintech,Indonesia,"Accel, Y Combinator, Amasia",2021,7
1099,Ajaib,Asia,2018,$245M,Jakarta,$1,10/4/2021,Fintech,Indonesia,"Softbank Ventures Asia, Alpha JWC Ventures, In...",2021,3
1144,Kopi Kenangan,Asia,2017,$333M,Jakarta,$1,12/27/2021,Consumer & retail,Indonesia,"Horizons Ventures, Sequoia Capital India, Alph...",2021,4


In [29]:
# Querying Industries with "Artificial intelligence"
filtered_df = merged_df.query("Industry == 'Artificial intelligence' and Continent == 'Asia' or Industry == 'Artificial Intelligence' and Continent == 'Asia'")
filtered_df

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn
137,Horizon Robotics,Asia,2015,$2B,Beijing,$5,2/27/2019,Artificial intelligence,China,"Hillhouse Capital Management, Linear Venture, ...",2019,4
193,MEGVII,Asia,2011,$1B,Beijing,$4,10/31/2017,Artificial intelligence,China,"Ant Financial Services Group, Russia-China Inv...",2017,6
417,Hesai Tech,Asia,2013,$717M,Shanghai,$2.18,8/1/2020,Artificial intelligence,China,"Lightspeed China Partners, Baidu Ventures, Qim...",2020,7
418,YITU Technology,Asia,2012,$401M,Shanghai,$2.17,3/8/2018,Artificial intelligence,China,"Sequoia Capital China, Banyan Capital",2018,6
442,Preferred Networks,Asia,2014,$147M,Tokyo,$2,5/17/2018,Artificial intelligence,Japan,"Toyota Motor Corporation, Mizuho Financial Gro...",2018,4
444,4Paradigm,Asia,2015,$1B,Beijing,$2,12/19/2018,Artificial intelligence,China,"Sequoia Capital China, China Construction Bank...",2018,3
473,XtalPi,Asia,2014,$786M,Shenzhen,$2,8/11/2021,Artificial intelligence,China,"Tencent Holdings, 5Y Capital, Sequoia Capital ...",2021,7
481,Black Sesame Technologies,Asia,2016,$115M,Shanghai,$2,9/22/2021,Artificial intelligence,China,"Northern Light Venture Capital, Xiaomi, Future...",2021,5
482,Advance Intelligence Group,Asia,2016,$536M,NaN,$2,9/23/2021,Artificial intelligence,Singapore,"Vision Plus Capital, GSR Ventures, ZhenFund",2021,5
495,Trax,Asia,2010,$1B,NaN,$2,7/22/2019,Artificial intelligence,Singapore,"Hopu Investment Management, Boyu Capital, DC T...",2019,9


In [30]:
group_summary = filtered_df['YearsToUnicorn'].agg(['min', 'max', 'mean', 'sum'])
group_summary

min       1.00
max       9.00
mean      4.69
sum     122.00
Name: YearsToUnicorn, dtype: float64

In [31]:
# Current columns with missing values:
# Missing values in 'Continent' column: 164 | Percentage of missing values: 13.28 %
# Missing values in 'YearFounded' column: 164 | Percentage of missing values: 13.28 %
# Missing values in 'Funding' column: 164 | Percentage of missing values: 13.28 %
# Missing values in 'City' column: 18 | Percentage of missing values: 1.46 %

# Solutions:
# NaN 'YearFounded' column will be filled with the mean of said column to replace all the NaN values
# NaN 'YearsToUnicorn' column will be filled after 'YearFounded' column have been filled
# NaN 'Funding' column will be transformed into a float to calculate the mean, before puting the values back in as well as adding a new column 'Funding_Numeric'
# 'City' and 'Continent' is left as is for now.

In [32]:
# Checking datatypes
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235 entries, 0 to 1239
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1235 non-null   object
 1   Continent        1071 non-null   object
 2   YearFounded      1071 non-null   Int64 
 3   Funding          1071 non-null   object
 4   City             1217 non-null   object
 5   Valuation($B)    1235 non-null   object
 6   DateJoined       1235 non-null   object
 7   Industry         1235 non-null   object
 8   Country          1235 non-null   object
 9   SelectInvestors  1235 non-null   object
 10  YearJoined       1235 non-null   int64 
 11  YearsToUnicorn   1071 non-null   object
dtypes: Int64(1), int64(1), object(10)
memory usage: 126.6+ KB


In [33]:
# YearFounded_Mean
YearFounded_Mean = int(merged_df["YearFounded"].mean())
print("Mean of year companies are found =", YearFounded_Mean)

Mean of year companies are found = 2012


In [34]:
merged_df["YearFounded"] = merged_df["YearFounded"].fillna(YearFounded_Mean)

In [35]:
# Checking datatypes, NaN values in "YearFounded" has been replaced with the mean of the column 
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235 entries, 0 to 1239
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1235 non-null   object
 1   Continent        1071 non-null   object
 2   YearFounded      1235 non-null   Int64 
 3   Funding          1071 non-null   object
 4   City             1217 non-null   object
 5   Valuation($B)    1235 non-null   object
 6   DateJoined       1235 non-null   object
 7   Industry         1235 non-null   object
 8   Country          1235 non-null   object
 9   SelectInvestors  1235 non-null   object
 10  YearJoined       1235 non-null   int64 
 11  YearsToUnicorn   1071 non-null   object
dtypes: Int64(1), int64(1), object(10)
memory usage: 126.6+ KB


In [36]:
# NaN values in "YearFounded" has been replaced with the mean of the column 
merged_df.head()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn
0,ByteDance,NaN,2012,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S...",2017,<NA>
1,SpaceX,North America,2002,$7B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012,10
2,SHEIN,Asia,2008,$2B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China...",2018,10
3,Stripe,North America,2010,$2B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG",2014,4
4,Canva,Oceania,2012,$572M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat...",2018,6


In [37]:
# Recalculating companies 'YearsToUnicorn'
merged_df['YearsToUnicorn'] = merged_df.apply(calculate_years_to_unicorn, axis=1)
merged_df.head()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn
0,ByteDance,NaN,2012,NaN,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S...",2017,5
1,SpaceX,North America,2002,$7B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012,10
2,SHEIN,Asia,2008,$2B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China...",2018,10
3,Stripe,North America,2010,$2B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG",2014,4
4,Canva,Oceania,2012,$572M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat...",2018,6


In [38]:
# YearsToUnicorn columns filled after replacing YearsFounded with mean of said column
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235 entries, 0 to 1239
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1235 non-null   object
 1   Continent        1071 non-null   object
 2   YearFounded      1235 non-null   Int64 
 3   Funding          1071 non-null   object
 4   City             1217 non-null   object
 5   Valuation($B)    1235 non-null   object
 6   DateJoined       1235 non-null   object
 7   Industry         1235 non-null   object
 8   Country          1235 non-null   object
 9   SelectInvestors  1235 non-null   object
 10  YearJoined       1235 non-null   int64 
 11  YearsToUnicorn   1235 non-null   int64 
dtypes: Int64(1), int64(2), object(9)
memory usage: 126.6+ KB


In [39]:
merged_df["Funding"]

0           NaN
1           $7B
2           $2B
3           $2B
4         $572M
5           $2B
6           $3B
7           $3B
8           $2B
9           $7B
10          $2B
11          $4B
12          $2B
13          $4B
14        $918M
15          $5B
16        $476M
17          $4B
18        $105M
19        $979M
20          $3B
21          NaN
22        $294M
23        $490M
24        $734M
25        $427M
26          $1B
27        $400M
28          $2B
29          $1B
30          $1B
31         $14B
32          NaN
33        $799M
34        $800M
35          $5B
36        $629M
37          $2B
38          $1B
39        $376M
40        $447M
41          $5B
42          $1B
44        $564M
45          NaN
46          $5B
47          $2B
48        $691M
49          $4B
50          $1B
51        $497M
52        $343M
53          $2B
54          $1B
55          NaN
56        $572M
57          $1B
58        $594M
59          $2B
60        $679M
61        $912M
62          $4B
63      

In [40]:
# Define a function to extract numeric values
def extract_numeric_value(value):
    match = re.search(r'\$(\d+(\.\d+)?)([BM]?)', str(value))
    if match:
        number = float(match.group(1))
        suffix = match.group(3)
        if suffix == 'B':
            number *= 1e9  # Convert to billions
        elif suffix == 'M':
            number *= 1e6  # Convert to millions
        return number
    else:
        return None

# Applying the function to convert the values in Funding
merged_df['Funding_Numeric'] = merged_df['Funding'].apply(extract_numeric_value)

# Calculate the mean of the numeric values
mean_value = merged_df['Funding_Numeric'].mean()

# Use SimpleImputer to replace NaN values with the mean
imputer = SimpleImputer(strategy='constant', fill_value=mean_value)
merged_df['Funding_Numeric'] = imputer.fit_transform(merged_df[['Funding_Numeric']])

merged_df["Funding_Numeric"].head()

0    5.58e+08
1    7.00e+09
2    2.00e+09
3    2.00e+09
4    5.72e+08
Name: Funding_Numeric, dtype: float64

In [44]:
def reconvert_extracted_numeric_value(number):
    if np.isnan(number):
        return np.nan
    suffix = ''
    if number >= 1e9:
        number /= 1e9
        suffix = 'B'
    elif number >= 1e6:
        number /= 1e6
        suffix = 'M'
    return f"${number:.1f}{suffix}"

merged_df['Funding'] = merged_df['Funding_Numeric'].apply(reconvert_extracted_numeric_value)

merged_df["Funding"].head()

0    $557.6M
1      $7.0B
2      $2.0B
3      $2.0B
4    $572.0M
Name: Funding, dtype: object

In [45]:
merged_df.head()

,Company,Continent,YearFounded,Funding,City,Valuation($B),DateJoined,Industry,Country,SelectInvestors,YearJoined,YearsToUnicorn,Funding_Numeric
0,ByteDance,NaN,2012,$557.6M,Beijing,$140,4/7/2017,Artificial intelligence,China,"Sequoia Capital China, SIG Asia Investments, S...",2017,5,5.58e+08
1,SpaceX,North America,2002,$7.0B,Hawthorne,$127,12/1/2012,Other,United States,"Founders Fund, Draper Fisher Jurvetson, Rothen...",2012,10,7.00e+09
2,SHEIN,Asia,2008,$2.0B,Shenzhen,$100,7/3/2018,E-commerce & direct-to-consumer,China,"Tiger Global Management, Sequoia Capital China...",2018,10,2.00e+09
3,Stripe,North America,2010,$2.0B,San Francisco,$95,1/23/2014,Fintech,United States,"Khosla Ventures, LowercaseCapital, capitalG",2014,4,2.00e+09
4,Canva,Oceania,2012,$572.0M,Surry Hills,$40,1/8/2018,Internet software & services,Australia,"Sequoia Capital China, Blackbird Ventures, Mat...",2018,6,5.72e+08


In [52]:
# Realigning column order for better visualization
# new_column_order = ['Company', 'Industry', 'Continent', 'Country', 'City', 'YearFounded', 'YearJoined', 'YearsToUnicorn', 'Funding', 'Funding_Numeric', 'Valuation($B)', 'SelectInvestors']
merged_df = merged_df.sort_index(axis=1)

In [53]:
# Final Result
merged_df.head()

,City,Company,Continent,Country,Funding,Funding_Numeric,Industry,SelectInvestors,Valuation($B),YearFounded,YearJoined,YearsToUnicorn
0,Beijing,ByteDance,NaN,China,$557.6M,5.58e+08,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",$140,2012,2017,5
1,Hawthorne,SpaceX,North America,United States,$7.0B,7.00e+09,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",$127,2002,2012,10
2,Shenzhen,SHEIN,Asia,China,$2.0B,2.00e+09,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China...",$100,2008,2018,10
3,San Francisco,Stripe,North America,United States,$2.0B,2.00e+09,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",$95,2010,2014,4
4,Surry Hills,Canva,Oceania,Australia,$572.0M,5.72e+08,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",$40,2012,2018,6
